# PyTorch Dataloader for the International Tree-Ring Data Bank (ITRDB)

This data works by wrapping our parsed ITRDB data in a PyTorch Dataset to be used for PyTorch Neural Networks. The parsed data is publicly hosted on an AWS S3 bucket, and is retrieved simply through the Python requests library. The Dataset will also cache the created dataframe, so the API request will only need to be made once per session, enabling you to create multiple Datasets (train, test, and validate) with little to no wait time. For the sake of simplicity, this Dataset will also limit the tree ring widths to between the years 1900-2023, and will then drop any rings that have 0 measurements between that time (row is all NaN between 1900-2023)

Import necessary dependencies

In [48]:
import torch
import requests
import pandas as pd
from io import StringIO
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

The Dataset Class itself, we retrieve the data from S3, cache it, and write a split function that can produce a 70-15-15 split on the data, only saving the set type requested in the constructor's parameters:

In [74]:
class TreeRingDataset(Dataset):

  _cache = None

  def __init__(self, set_type="train"):

    if TreeRingDataset._cache is None:
      res = requests.get("https://paleo-data.s3.amazonaws.com/data.csv")
      TreeRingDataset._cache = pd.read_csv(StringIO(res.text), sep=",")

    self.df = TreeRingDataset._cache.copy()

    self.df.drop(self.df.columns[list(range(1, 1940))], axis=1, inplace=True)
    self.df.dropna(subset=self.df.columns[1:125], how='all', inplace=True)

    training, test, validate = self.__split()
    self.df = training if set_type == "train" else (test if set_type == "test" else validate)

  def __split(self):
    train, test = train_test_split(self.df, train_size=.70, stratify=self.df['loc'])
    validate, test = train_test_split(test, train_size=.5, stratify=test['loc'])
    return (train, test, validate)

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):
    x = torch.tensor(self.df.iloc[index, 1:52])
    latn, lats, lone, lonw = self.df.iloc[index, 87:91]
    label = torch.tensor([(latn+lats)/2, (lone+lonw)/2])

    return x, label

In [62]:
a = TreeRingDataset(set_type="train")

Then this can be taken and retrieved for each train, test, and validate set, which will only need to send 1 API request and thus finish fairly quickly:

In [75]:
train_data = TreeRingDataset(
    set_type="train"
)

test_data = TreeRingDataset(
    set_type="test"
)

validate_data = TreeRingDataset(
    set_type="validate"
)

Finally, we can wrap it in a Dataloader:

In [76]:
train_dataloader = DataLoader(train_data, batch_size=32)
test_dataloader = DataLoader(test_data, batch_size=32)
validate_dataloader = DataLoader(validate_data, batch_size=32)

And just an example of what the data looks like, we can display the first batch:

In [77]:
train_features, train_labels = next(iter(train_dataloader))
print(train_features)
print(train_labels)

tensor([[0.5100, 0.7900, 0.7600,  ..., 0.8200, 0.3400,    nan],
        [0.6200, 0.7400, 0.7300,  ...,    nan,    nan,    nan],
        [   nan,    nan,    nan,  ...,    nan,    nan,    nan],
        ...,
        [   nan,    nan,    nan,  ...,    nan,    nan,    nan],
        [0.3200, 0.2200, 0.3400,  ...,    nan,    nan,    nan],
        [0.6000, 0.8200, 0.4700,  ...,    nan,    nan,    nan]],
       dtype=torch.float64)
tensor([[  35.5833, -106.1667],
        [  39.3700, -114.7200],
        [  35.4000, -108.5167],
        [  38.6000, -107.5800],
        [  34.9200, -107.8300],
        [  42.5300,  -72.6000],
        [  38.4000, -105.3000],
        [  37.6500, -112.6700],
        [  24.7140,  -81.3850],
        [  39.6700, -107.8800],
        [  42.5300,  -72.6000],
        [  46.8300,  -87.9300],
        [  38.8300, -108.5700],
        [  44.9140, -109.5730],
        [  24.7140,  -81.3850],
        [  31.2200,  -84.4800],
        [  40.0300, -105.5800],
        [  24.7140,  -81.3850]